In [47]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
import numpy as np

In [48]:
#Pull data from csv file- traffic camera locations
csv_file = "../Desktop/traffic-cameras-1.csv"
traffic_cameras_df = pd.read_csv(csv_file)
traffic_cameras_df["Longitude"] = traffic_cameras_df["Longitude"].map("{:.3f}".format)
traffic_cameras_df["Latitude"] = traffic_cameras_df["Latitude"].map("{:.3f}".format)
traffic_cameras_df.head()

,Camera ID,Camera Manufacturer,Turn on Date,ATD Location ID,Primary St Segment ID,Cross St Segment ID,Landmark,Signal Engineer Area,Council District,JURISDICTION,...,CROSS_ST_AKA,Cross Street Block,PRIMARY_ST_AKA,Location,IP Comm Status,IP Comm Status Date and Time,Longitude,Latitude,Published Screenshots,Screenshot Address
0,68,Sarix,NaN,LOC16-000180,2017636.0,2017637.0,NaN,CENTRAL,9,NaN,...,NaN,401.0,NaN,"(30.2837963, -97.7419052)",OFFLINE,06/23/2017 01:59:00 AM +0000,-97.742,30.284,NaN,http://162.89.4.145/CCTVImages/CCTV68.jpg
1,1,Sarix,11/18/2016,LOC16-005090,3268115.0,NaN,"Little Walnut Creek Library, HEB",NORTHEAST,4,NaN,...,NaN,NaN,NaN,"(30.3636856, -97.6981583)",OFFLINE,10/02/2017 05:18:00 PM +0000,-97.698,30.364,NaN,http://162.89.4.145/CCTVImages/CCTV1.jpg
2,3,Sarix,12/20/2016,LOC16-004915,5102830.0,2029964.0,NaN,NORTHWEST,6,NaN,...,NaN,10001.0,NaN,"(30.4819565, -97.7895737)",OFFLINE,05/11/2017 05:00:00 AM +0000,-97.790,30.482,NaN,http://162.89.4.145/CCTVImages/CCTV3.jpg
3,4,Spectra Enhanced,11/30/2016,LOC16-003630,2040576.0,2020020.0,NaN,CENTRAL,10,NaN,...,NaN,3400.0,NaN,"(30.31354, -97.76716)",OFFLINE,05/11/2017 05:00:00 AM +0000,-97.767,30.314,NaN,http://162.89.4.145/CCTVImages/CCTV4.jpg
4,2,Spectra Enhanced,11/08/2016,LOC16-000805,2018548.0,2018299.0,NaN,NORTHEAST,"3,1",NaN,...,NaN,3300.0,NaN,"(30.2735558, -97.6979065)",ONLINE,10/02/2017 08:46:00 PM +0000,-97.698,30.274,NaN,http://162.89.4.145/CCTVImages/CCTV2.jpg


In [49]:
cam_columns = ["Camera ID", "Turn on Date", "Location Name", "Location","Longitude","Latitude"]
new_traffic_cameras_df = traffic_cameras_df[cam_columns].copy()
new_traffic_cameras_df.head()

,Camera ID,Turn on Date,Location Name,Location,Longitude,Latitude
0,68,NaN,GUADALUPE ST / 21ST ST,"(30.2837963, -97.7419052)",-97.742,30.284
1,1,11/18/2016,830 BLK W RUNDBERG LN (Little Walnut Creek Lib...,"(30.3636856, -97.6981583)",-97.698,30.364
2,3,12/20/2016,STAKED PLAINS DR / LAKELINE BLVD,"(30.4819565, -97.7895737)",-97.790,30.482
3,4,11/30/2016,35TH ST / PECOS ST,"(30.31354, -97.76716)",-97.767,30.314
4,2,11/08/2016,AIRPORT BLVD / OAK SPRINGS DR,"(30.2735558, -97.6979065)",-97.698,30.274


In [50]:
new_traffic_cameras_df.dtypes

Camera ID         int64
Turn on Date     object
Location Name    object
Location         object
Longitude        object
Latitude         object
dtype: object

In [55]:
#Pull data from csv file- traffic incidents
csv_file = "../Desktop/Real-Time_Traffic_Incident_Reports.csv"
traffic_incidents_df = pd.read_csv(csv_file)
traffic_incidents_df["Longitude"] = traffic_incidents_df["Longitude"].map("{:.3f}".format)
traffic_incidents_df["Latitude"] = traffic_incidents_df["Latitude"].str.replace(',','').astype(np.float64)
traffic_incidents_df["Latitude"] = traffic_incidents_df["Latitude"].map("{:.3f}".format)
traffic_incidents_df.dropna()
traffic_incidents_df.head()

,Traffic Report ID,Published Date,Issue Reported,Location,Latitude,Longitude,Address,Status,Status Date
0,C163BCD1CF90C984E9EDA4DBA311BCA369A7D1A1_15288...,06/13/2018 06:35:59 AM +0000,Crash Service,"(30.283797,-97.741906)",30.284,-97.742,W 21ST ST & GUADALUPE ST,ARCHIVED,06/13/2018 09:00:03 AM +0000
1,2664C206999E7419517EE39E177797576A52201B_15381...,09/29/2018 12:52:58 AM +0000,Traffic Hazard,"(30.380525,-97.737873)",30.381,-97.738,0 Mopac Sb To Research Sb Ramp,ARCHIVED,09/29/2018 01:35:03 AM +0000
2,5BB246A3223A89C7FB9B2EED9A2EBDA82141406D_15438...,12/03/2018 05:49:02 PM +0000,Crash Service,"(30.278372,-97.671971)",30.278,-97.672,5800 Techni Center Dr,ARCHIVED,12/03/2018 06:30:03 PM +0000
3,6B20382196FB454E9FD06A33E60142902A2F0706_15288...,06/13/2018 10:15:36 AM +0000,Traffic Hazard,"(30.339593,-97.700963)",30.340,-97.701,400-717 E ANDERSON LN EB,ARCHIVED,06/13/2018 11:20:03 AM +0000
4,5CB97D4F03F917B5DCE1B3F99B1EB647FF58DA7F_15521...,03/09/2019 03:07:05 PM +0000,Traffic Hazard,"(30.201332,-97.77585)",30.201,-97.776,6000 S Congress Ave,ARCHIVED,03/09/2019 03:15:04 PM +0000


In [56]:
incident_columns = ["Published Date", "Issue Reported","Location", "Longitude","Latitude", "Address"]
new_traffic_incidents_df = traffic_incidents_df[incident_columns].copy()
new_traffic_incidents_df.dropna()
new_traffic_incidents_df.head()

,Published Date,Issue Reported,Location,Longitude,Latitude,Address
0,06/13/2018 06:35:59 AM +0000,Crash Service,"(30.283797,-97.741906)",-97.742,30.284,W 21ST ST & GUADALUPE ST
1,09/29/2018 12:52:58 AM +0000,Traffic Hazard,"(30.380525,-97.737873)",-97.738,30.381,0 Mopac Sb To Research Sb Ramp
2,12/03/2018 05:49:02 PM +0000,Crash Service,"(30.278372,-97.671971)",-97.672,30.278,5800 Techni Center Dr
3,06/13/2018 10:15:36 AM +0000,Traffic Hazard,"(30.339593,-97.700963)",-97.701,30.340,400-717 E ANDERSON LN EB
4,03/09/2019 03:07:05 PM +0000,Traffic Hazard,"(30.201332,-97.77585)",-97.776,30.201,6000 S Congress Ave


In [57]:
new_traffic_incidents_df.dtypes

Published Date    object
Issue Reported    object
Location          object
Longitude         object
Latitude          object
Address           object
dtype: object